# Heatmap SPI-1 Wilayah Lembang – Juni 2020 (dengan Label Min/Max)
Visualisasi spasial indeks kekeringan SPI-1 berdasarkan data bulan Juni 2020 di wilayah Lembang.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import matplotlib.path as mpath


## 1. Load Data

In [ ]:

df = pd.read_csv("Lembang_Merged_SPI_CLEANED.csv")
boundary_df = pd.read_csv("Koordinat_Polygon_Lembang.csv")


## 2. Filter Data SPI-1 Bulan Juni 2020

In [ ]:

df["date"] = pd.to_datetime(df["date"])
spi1_jun = df[df["date"] == "2020-06-01"]
spi1_jun = spi1_jun.groupby(["lat", "lon"])["SPI_1"].mean().reset_index()


## 3. Interpolasi dan Masking Wilayah

In [ ]:

grid_lon = np.linspace(spi1_jun["lon"].min(), spi1_jun["lon"].max(), 300)
grid_lat = np.linspace(spi1_jun["lat"].min(), spi1_jun["lat"].max(), 300)
grid_lon, grid_lat = np.meshgrid(grid_lon, grid_lat)
grid_z = griddata((spi1_jun["lon"], spi1_jun["lat"]), spi1_jun["SPI_1"], (grid_lon, grid_lat), method='linear')

boundary_coords = boundary_df[['lon', 'lat']].values
polygon_path = mpath.Path(boundary_coords)
points = np.vstack((grid_lon.flatten(), grid_lat.flatten())).T
mask = polygon_path.contains_points(points).reshape(grid_lon.shape)
masked_grid_z = np.where(mask, grid_z, np.nan)


## 4. Visualisasi Heatmap SPI-1 dengan Titik Min/Max

In [ ]:

min_spi1_val = spi1_jun["SPI_1"].min()
max_spi1_val = spi1_jun["SPI_1"].max()
min_spi1_loc = spi1_jun[spi1_jun["SPI_1"] == min_spi1_val][["lon", "lat"]].values[0]
max_spi1_loc = spi1_jun[spi1_jun["SPI_1"] == max_spi1_val][["lon", "lat"]].values[0]

plt.figure(figsize=(10, 10))
contour = plt.contourf(
    grid_lon, grid_lat, masked_grid_z,
    levels=100, cmap="RdYlBu_r", alpha=0.95
)
plt.colorbar(contour, label="SPI-1")
plt.contour(grid_lon, grid_lat, masked_grid_z, levels=15, linewidths=0.4, colors='black', alpha=0.3)
plt.plot(boundary_coords[:, 0], boundary_coords[:, 1], color="white", linewidth=1.5)

# Anotasi titik minimum dan maksimum
plt.plot(*min_spi1_loc, marker='o', color='black')
plt.text(min_spi1_loc[0], min_spi1_loc[1], f"Min: {min_spi1_val:.2f}", fontsize=10, ha='right', va='bottom', color='black')

plt.plot(*max_spi1_loc, marker='o', color='black')
plt.text(max_spi1_loc[0], max_spi1_loc[1], f"Max: {max_spi1_val:.2f}", fontsize=10, ha='left', va='top', color='black')

plt.grid(True, which='major', linestyle='--', linewidth=0.5, alpha=0.4)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Heatmap SPI-1 – Bulan Juni 2020 (dengan Label Min/Max)", fontsize=13)
plt.gca().set_facecolor("white")
plt.axis("equal")
plt.tight_layout()
plt.show()
